In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from builtins import input

#import system things
# from tensorflow.examples.tutorials.mnist import input_data # for data
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import offsetbox
import os


In [2]:



def visualize(embed, x_test, y_test):

    # two ways of visualization: scale to fit [0,1] scale
    # feat = embed - np.min(embed, 0)
    # feat /= np.max(feat, 0)

    # two ways of visualization: leave with original scale
    feat = embed
    ax_min = np.min(embed,0)
    ax_max = np.max(embed,0)
    ax_dist_sq = np.sum((ax_max-ax_min)**2)

    plt.figure()
    ax = plt.subplot(111)
    colormap = plt.get_cmap('tab10')
    shown_images = np.array([[1., 1.]])
    for i in range(feat.shape[0]):
        dist = np.sum((feat[i] - shown_images)**2, 1)
        if np.min(dist) < 3e-4*ax_dist_sq:   # don't show points that are too close
            continue
        shown_images = np.r_[shown_images, [feat[i]]]
        patch_to_color = np.expand_dims(x_test[i], -1)
        patch_to_color = np.tile(patch_to_color, (1, 1, 3))
        patch_to_color = (1-patch_to_color) * (1,1,1) + patch_to_color * colormap(y_test[i]/10.)[:3]
        imagebox = offsetbox.AnnotationBbox(
            offsetbox.OffsetImage(patch_to_color, zoom=0.5, cmap=plt.cm.gray_r),
            xy=feat[i], frameon=False
        )
        ax.add_artist(imagebox)

    plt.axis([ax_min[0], ax_max[0], ax_min[1], ax_max[1]])
    # plt.xticks([]), plt.yticks([])
    plt.title('Embedding from the last layer of the network')
    plt.show()

In [6]:
class Siamese:

    # Create model
    def __init__(self):
        self.x1 = tf.placeholder(tf.float32, [None, 784])
        self.x2 = tf.placeholder(tf.float32, [None, 784])

        with tf.variable_scope("siamese", reuse=tf.AUTO_REUSE) as scope:
            self.o1 = self.network(self.x1)
            scope.reuse_variables()
            self.o2 = self.network(self.x2)

        # Create loss
        self.y_ = tf.placeholder(tf.float32, [None])
        self.loss = self.loss_with_spring()

    def network(self, x):
        weights = []
        fc1 = self.fc_layer(x, 1024, "fc1")
        ac1 = tf.nn.relu(fc1)
        fc2 = self.fc_layer(ac1, 1024, "fc2")
        ac2 = tf.nn.relu(fc2)
        fc3 = self.fc_layer(ac2, 2, "fc3")
        return fc3

    def fc_layer(self, bottom, n_weight, name):
        assert len(bottom.get_shape()) == 2
        n_prev_weight = bottom.get_shape()[1]
        initer = tf.truncated_normal_initializer(stddev=0.01)
        W = tf.get_variable(name+'W', dtype=tf.float32, shape=[n_prev_weight, n_weight], initializer=initer)
        b = tf.get_variable(name+'b', dtype=tf.float32, initializer=tf.constant(0.01, shape=[n_weight], dtype=tf.float32))
        fc = tf.nn.bias_add(tf.matmul(bottom, W), b)
        return fc

    def loss_with_spring(self):
        margin = 5.0
        labels_t = self.y_
        labels_f = tf.subtract(1.0, self.y_, name="1-yi")          # labels_ = !labels;
        eucd2 = tf.pow(tf.subtract(self.o1, self.o2), 2)
        eucd2 = tf.reduce_sum(eucd2, 1)
        eucd = tf.sqrt(eucd2+1e-6, name="eucd")
        C = tf.constant(margin, name="C")
        # yi*||CNN(p1i)-CNN(p2i)||^2 + (1-yi)*max(0, C-||CNN(p1i)-CNN(p2i)||^2)
        pos = tf.multiply(labels_t, tf.pow(eucd,2), name="yi_x_eucd2")
#         neg = tf.multiply(labels_f, tf.subtract(0.0,eucd2), name="yi_x_eucd2")
#         neg = tf.multiply(labels_f, tf.maximum(0.0, tf.subtract(C,eucd2)), name="Nyi_x_C-eucd_xx_2")
        neg = tf.multiply(labels_f, tf.pow(tf.maximum(tf.subtract(C, eucd), 0.0), 2), name="Nyi_x_C-eucd_xx_2")
        losses = tf.add(pos, neg, name="losses")
        loss = tf.reduce_mean(losses, name="loss")
        return loss

    def loss_with_step(self):
        margin = 5.0
        labels_t = self.y_
        labels_f = tf.subtract(1.0, self.y_, name="1-yi")          # labels_ = !labels;
        eucd2 = tf.pow(tf.subtract(self.o1, self.o2), 2)
        eucd2 = tf.reduce_sum(eucd2, 1)
        eucd = tf.sqrt(eucd2+1e-6, name="eucd")
        C = tf.constant(margin, name="C")
        pos = tf.multiply(labels_t, eucd, name="y_x_eucd")
        neg = tf.multiply(labels_f, tf.maximum(0.0, tf.subtract(C, eucd)), name="Ny_C-eucd")
        losses = tf.add(pos, neg, name="losses")
        loss = tf.reduce_mean(losses, name="loss")
        return loss

In [4]:
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
batch_size = 128
def shuffle_in_unison(a, b):
    rng_state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(rng_state)
    np.random.shuffle(b)

shuffle_in_unison(np.copy(train_images), np.copy(train_labels))
shuffle_in_unison(np.copy(test_images), np.copy(test_labels))

def get_train_mini_batch(iter_num):
    """
    Args:
        iter_num: iteration number
    Returns:
        Tuple of training images and labels
    """
    start = (iter_num * batch_size)% train_images.shape[0]
    end = min(start + batch_size, train_images.shape[0])
    images = train_images[start:end]
    images = images.reshape([images.shape[0], 784])
    
    labels = np.zeros([end-start, 10])
    label_ints = train_labels[start:end].astype(int)
#     labels[np.arange(end-start), label_ints] = 1
    
    return (images, label_ints)

def get_test_mini_batch(iter_num):
    """
    Args:
        iter_num: iteration number
    Returns:
        Tuple of training images and labels
    """
    start = (iter_num * batch_size)% test_images.shape[0]
    end = min(start + batch_size, test_images.shape[0])
    images = test_images[start:end]
    images = images.reshape([images.shape[0], 784])
    
    labels = np.zeros([end-start, 10])
    label_ints = test_labels[start:end].astype(int)
    labels[np.arange(end-start), label_ints] = 1
    
    return (images, label_ints)

In [7]:

# mnist = input_data.read_data_sets('MNIST_data', one_hot=False)


# setup siamese network
siamese = Siamese()
with tf.variable_scope("Adam", reuse=tf.AUTO_REUSE) as scope:
    train_step = tf.train.AdamOptimizer(1e-4).minimize(siamese.loss)
saver = tf.train.Saver()

# if you just want to load a previously trainmodel?
load = False
model_ckpt = './model.meta'
if os.path.isfile(model_ckpt):
    input_var = None
    while input_var not in ['yes', 'no']:
        input_var = input("We found model files. Do you want to load it and continue training [yes/no]?")
    if input_var == 'yes':
        load = True
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    # start training
    if load: saver.restore(sess, './model')

    for step in range(20000):
        batch_x1, batch_y1 = get_train_mini_batch(np.random.randint(50000))
        batch_x2, batch_y2 = get_train_mini_batch(np.random.randint(50000))
        min_batch_size = min([batch_x1.shape[0], batch_x2.shape[0]])
        batch_x1, batch_y1 = batch_x1[:min_batch_size, :], batch_y1[:min_batch_size]
        batch_x2, batch_y2 = batch_x2[:min_batch_size, :], batch_y2[:min_batch_size]
        batch_y = (batch_y1 == batch_y2).astype('float')
        _, loss_v = sess.run([train_step, siamese.loss], feed_dict={
                            siamese.x1: batch_x1,
                            siamese.x2: batch_x2,
                            siamese.y_: batch_y})

        if np.isnan(loss_v):
            print('Model diverged with loss = NaN')
            quit()

        if step % 10 == 0:
            print ('step %d: loss %.3f' % (step, loss_v))

        if step % 10 == 0 and step > 0:
            saver.save(sess, './model')
            x_test, y_test = get_test_mini_batch(step)
            embed = siamese.o1.eval({siamese.x1: x_test})
            embed.tofile('embed.txt')

    # visualize result
    x_test, y_test = get_test_mini_batch(1)
#     x_test = mnist.test.images.reshape([-1, 28, 28])
#     y_test = mnist.test.labels
    visualize(embed, x_test, y_test)

We found model files. Do you want to load it and continue training [yes/no]?no
step 0: loss 15.868
step 10: loss 3.340
step 20: loss 2.820
step 30: loss 3.512
step 40: loss 2.216
step 50: loss 1.545
step 60: loss 2.737
step 70: loss 2.227
step 80: loss 2.552
step 90: loss 3.018
step 100: loss 2.118
step 110: loss 1.508
step 120: loss 2.563
step 130: loss 1.650
step 140: loss 1.880
step 150: loss 2.550
step 160: loss 1.858


KeyboardInterrupt: 